In [1]:
import pandas as pd 
import numpy as np 
import os
from os import listdir
from os.path import isfile, join

In [28]:
path = '/Users/jones/Downloads/P19_48_72_images/results_filt/72hrs/aorta03'
duration = '72hr'
aorta = 'aorta03'

In [29]:
singletonDistances = []
withinClusterDistances = []
betweenClusterDistances = []

os.chdir(path)
files = [f for f in listdir(path) if isfile(join(path, f)) if ".DS_Store" not in f if ".csv" in f if "._" not in f]
for file in files:

    ###Singletons
    df_help = pd.read_csv(join(path, file))
    unique, counts = np.unique(df_help['Counter'], return_counts=True)
    singletons = unique[counts == 1]
    for s in singletons:
        xSingleton = df_help.loc[df_help['Counter'] == s]['X'].values[0]
        ySingleton = df_help.loc[df_help['Counter'] == s]['Y'].values[0]
        df_other = df_help.loc[df_help['Counter'] != s]
        xother = df_other['X'].values
        yother = df_other['Y'].values
        points = np.column_stack((xother, yother))
        distancesSingleton = np.linalg.norm(points - [xSingleton,ySingleton], axis=1)
        singletonDistances.append(distancesSingleton[np.argmin(distancesSingleton)])

    ### distances between all cluster
    higher = unique[counts != 1]
    for h in higher:
        xvalsCluster = df_help.loc[df_help['Counter'] == h]['X'].values
        yvalsCluster = df_help.loc[df_help['Counter'] == h]['Y'].values
        pointsCluster = np.column_stack((xvalsCluster, yvalsCluster))
        df_other = df_help.loc[df_help['Counter'] != h]
        if len(df_other != 0):
            xother = df_other['X'].values
            yother = df_other['Y'].values
            pointsOther = np.column_stack((xother, yother))
            minClusterDistance = 9999999999
            for i, point in enumerate(pointsCluster):
                distancesPoint = np.linalg.norm(pointsOther - point, axis=1)
                if minClusterDistance > np.min(distancesPoint[distancesPoint > 0]):
                    minClusterDistance = np.min(distancesPoint[distancesPoint > 0])
            betweenClusterDistances.append(minClusterDistance)

    #normalization
    dfBiggerCluster = df_help[~df_help['Counter'].isin(singletons)]
    min_distances = []
    for i in np.unique(dfBiggerCluster['Counter']):
        dfCluster = df_help.loc[df_help['Counter'] == i]
        xCluster = dfCluster['X'].values
        yCluster = dfCluster['Y'].values
        points = np.column_stack((xCluster, yCluster))
        for i, point in enumerate(points):
            distances = np.linalg.norm(points - point, axis=1)
            withinClusterDistances.append(np.min(distances[distances > 0]))
        


In [30]:
normalizedSingleton = singletonDistances/np.mean(withinClusterDistances)
normalizedWithinCluster = withinClusterDistances/np.mean(withinClusterDistances)
normalizedBetweenCluster = betweenClusterDistances/np.mean(withinClusterDistances)

In [34]:
df_singleton = pd.DataFrame({'duration': duration, 'aorta': aorta, 'type': 'singleton', 'normalizedDistance': normalizedSingleton})
df_withinCluster = pd.DataFrame({'duration': duration, 'aorta': aorta, 'type': 'withinCluster', 'normalizedDistance': normalizedWithinCluster})
df_betweenCluster = pd.DataFrame({'duration': duration, 'aorta': aorta, 'type': 'betweenCluster', 'normalizedDistance': normalizedBetweenCluster})

In [35]:
dfAll = pd.concat([df_singleton, df_withinCluster, df_betweenCluster])

In [36]:
dfAll.to_csv('/Users/jones/Downloads/normalizedDistances_72hr_aorta03.csv', index=False)

In [48]:
### concat distances

In [50]:
df1 = pd.read_csv('/Users/jones/Downloads/normalizedDistances_72hr_aorta01.csv')
df2 = pd.read_csv('/Users/jones/Downloads/normalizedDistances_72hr_aorta02.csv')
df3 = pd.read_csv('/Users/jones/Downloads/normalizedDistances_72hr_aorta03.csv')

dfAllAge = pd.concat([df1, df2, df3])
dfAllAge.to_csv('/Users/jones/Downloads/normalizedDistances_72hr.csv', index=False)